In [1]:
from typing import Callable, Dict, List, Tuple, Union
import numpy as np
import numpy.random as rn
from itertools import product
from importlib import reload
import coursework as cw

In [2]:
m = 3
transpose_rows = []

n_binary_encodings = [np.binary_repr(i, width=m) for i in range(1,2**m)]
n_binary_encodings = [[int(c[i]) for i in range(m)] for c in n_binary_encodings]

for i in range(m-1, -1, -1):
    transpose_rows.append([n_binary_encodings[j][i] for j in range(2**m-1)])

H = np.array(transpose_rows)
H, H.shape

# np.array(matrix_rows)

(array([[1, 0, 1, 0, 1, 0, 1],
        [0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1, 1]]),
 (3, 7))

In [3]:
m = 3
k = np.arange(4)
k_int = k.shape[0]

transpose_rows = []
parity_bit_positions = [((2**i)-1) for i in range(m)]
parity_idx = 0

identity_k = np.identity(k_int)
identity_idx = 0

for row_idx in range(k_int+m): 
    row = []
    if row_idx in parity_bit_positions:
        parity_bits_from_H = H[:][parity_idx]
        for bit in range(H.shape[1]):
            if bit not in parity_bit_positions:
                row.append(parity_bits_from_H[bit])
        parity_idx += 1
    else:
        row = [int(item) for item in identity_k[identity_idx].tolist()]
        identity_idx += 1

    transpose_rows.append(row)

G = np.array(transpose_rows).T
G


array([[1, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 0],
       [1, 1, 0, 1, 0, 0, 1]])

In [4]:
k = 4
m = 3
data = np.array([0, 1, 0, 1])

n = np.zeros(m+k)
n[4] = 1

t = (G.T @ data) % 2 # transmitted string
r = (t+n) % 2 # received string with flipped bit in position 5
z = (H@r) % 2

print(data,'\n', t,'\n',z)
if (z>0).any():
    z_binary = [int(i) for i in z]
    error_pos = 0
    for i in range(len(z_binary)):
        error_pos += (z_binary[i] * (2**i))
    print(f"Error detected in position {error_pos}")
    print(f"Received message: {r}")
    r[error_pos-1] = int(not r[error_pos-1])
    print(f"Corrected message: {r}")

R = np.copy(G)
parity_bits = [((2**i)-1) for i in range(m)]
for i in parity_bits:
    R[:,i] = np.zeros(R.shape[0]).T
decoded_r = (R@r) % 2
decoded_r

[0 1 0 1] 
 [0 1 0 0 1 0 1] 
 [1. 0. 1.]
Error detected in position 5
Received message: [0. 1. 0. 0. 0. 0. 1.]
Corrected message: [0. 1. 0. 0. 1. 0. 1.]


array([0., 1., 0., 1.])

In [132]:
from pandas.core.common import flatten

with open('secret.txt', 'r') as f:
    bits = f.read()
msg = np.array([ int(b) for b in bits ]).astype(int)

decoded_msg = {}
for m in range(1,11):
    n = 2**m - 1
    if len(msg) % n == 0:
        print(f"m={m}, n={n}")
        splits = len(msg) // n
        decoded_string = []
        i = 0
        done = False
        while not done:
            start = i*n
            end = start + n
            codeword = msg[start:end].copy()
            decoded_string.append((cw.hamming_decode(codeword, m)).tolist())
            i += 1
            if end + n > len(msg): 
                done = True
        decoded_msg[m] = decoded_string
    else: 
        print(f"m={m} not valid")
        continue

secrets = {}
for k in decoded_msg.keys():
    try:
        string_code = np.array(decoded_msg[k]).flatten()
        decoded_secret = cw.bits2text(string_code)
        secrets[k] = decoded_secret
    except:
        print(f"m={k} not valid")

m=1, n=1
m=2, n=3
m=3, n=7
m=4, n=15
m=5 not valid
m=6 not valid
m=7 not valid
m=8 not valid
m=9 not valid
m=10 not valid


In [135]:
alphabet = "abcdefghijklmnopqrstuvwxyz01234567890 .,\n"
alphabet_chars = [c[0] for c in alphabet]

for k in secrets.keys():
    test_word = secrets[k][:10]
    broken = False
    for c in test_word:
        if c not in alphabet_chars:
            broken = True
        if broken:
            break

    if not broken: 
        print(secrets[k])


the bell system technical journal
vol. xx, april, 1950, no. 2
error detecting and error correcting codes
by r. w. hamming
1. introduction
he author was"ìed to the studyªgiven in this paper from a consideration of large scale computing machines in which a large number of
operations must be performed without a single error in the end result- this
problem of dÏing things right on a large scale is not essentially new in a
telephone central office, for example, a very large nuiber of operations are
performed w(	le the errors leading to wrong numbers are kept wedl under
control, though they have not been completely eliminated. thkS has been

achieved, in part, through the use of selfchecking circuits. the occasional
failure that escapes routine checking is still detected by the customer and
will, if it persists, result in customer complaint, while if it is transient it will
produce only occasional wrong numbers. at the same time the rest of the
central office functions satisfactorily. in a 

In [133]:
alphabet = [c[0] for c in "abcdefghijklmnopqrstuvwxyz01234567890 .,\n"]

for k in secrets.keys():
    test_word = secrets[k][:10]
    broken = False
    for c in test_word:
        if c not in alphabet:
            broken = True
        if broken:
            break
        
    if not broken: 
        print(f"\nm={k}: {secrets[k]}")


m=1: 

m=4: the bell system technical journal
vol. xx, april, 1950, no. 2
error detecting and error correcting codes
by r. w. hamming
1. introduction
he author was"ìed to the studyªgiven in this paper from a consideration of large scale computing machines in which a large number of
operations must be performed without a single error in the end result- this
problem of dÏing things right on a large scale is not essentially new in a
telephone central office, for example, a very large nuiber of operations are
performed w(	le the errors leading to wrong numbers are kept wedl under
control, though they have not been completely eliminated. thkS has been

achieved, in part, through the use of selfchecking circuits. the occasional
failure that escapes routine checking is still detected by the customer and
will, if it persists, result in customer complaint, while if it is transient it will
produce only occasional wrong numbers. at the same time the rest of the
central office functions satisfact